<a href="https://colab.research.google.com/github/syphax/solar-data/blob/jan08-more-clean-data/nb/Clean_GMP_Solar_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro

This notebook pre-processes raw downloads from https://greenmountainpower.com/account/usage and produces one cleansed file, suitable for further analysis by the `Solar Viz` notebook.

To run this script, you need access to Google Drive, and you need to copy the data from https://github.com/syphax/solar-data/tree/main/data to `/My Drive/Data/Solar` (or edit the path variable in the 2nd code block to point somewhere else).

_TODO: Load the data directly from the GitHub repo._



# Setup

In [ ]:
import os
import re

from datetime import datetime

import numpy as np
import pandas as pd

import altair as alt
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# You can of course edit this to taste:

path = '/content/drive/MyDrive/Data/Solar/'

In [ ]:
# This will require you to click through a couple windows to 
# give permission to access your GDrive.

from google.colab import drive
drive.mount('/content/drive')

# Load Data

This script preps data that was downloaded from [Green Mountain Power's website](https://greenmountainpower.com/account/usage/).

GMP has an excellent UI for reporting usage, and provides downloadable data in 15 minute increments (either CSV or Green Button XML). *Unfortunately* it only supports manual data downloads in 15 day (max) chunks.  
*Fortunately* it only takes a couple minutes to download several months of data. The manual downloads are just easy enough that I haven't bothered to automate the process.

Fields in the CSV downloads are:
* `ServiceAgreement`: Account info. Format is `Account Holder / Service / Service Acronym / Account Start Date / Account Status`
* `IntervalStart`: Timestamp; format is `yyyy-MM-dd-hh:mm:ss`
* `IntervalEnd`: Same, 15 minutes later. Redundant but explicit!
* `Quantity`: Amount of electricity generated
* `UnitOfMeasure` `kWh`. I love that they have an explicit UoM field!


In [ ]:
path = '/content/drive/MyDrive/Data/Solar/'

raw_input_files = os.path.join(path, 'UsageData*.csv')
joined_input_file = os.path.join(path, 'full_dataset.csv')


In [ ]:
# This should list the data files that you copied from https://github.com/syphax/solar-data/tree/main/data

!ls $path 

In [ ]:
# This concatenates available data files. We will need to remove possible dupes, and check for completeness.

!cat $raw_input_files > $joined_input_file

In [ ]:
df_energy_data_raw = pd.read_csv(joined_input_file)

# Check and Clean

In [ ]:
# What fields are there?

df_energy_data_raw.dtypes

In [ ]:
# Make a clean field for kWh values

df_energy_data_raw['kWh'] = 0

df_energy_data_raw['kWh'] = np.where(df_energy_data_raw['UnitOfMeasure'] == 'kWh', df_energy_data_raw['Quantity'], 0)
df_energy_data_raw['kWh'] = df_energy_data_raw['kWh'].astype(np.float64)

In [ ]:
# Add date field for interval starts

fmt = '%Y-%m-%d-%H:%M:%S'
df_energy_data_raw['dt_start'] = pd.to_datetime(df_energy_data_raw['IntervalStart'], format=fmt, errors='coerce').dt.date

In [ ]:
# Quick check of values in df:

df_energy_data_raw.groupby(['ServiceAgreement', 'UnitOfMeasure'], as_index=False).agg(cnt_records=('ServiceAgreement','count'), 
                                                                                      unique_dt=('IntervalStart','nunique'), 
                                                                                      kwh=('kWh','sum'))

## Extract service level code

In [ ]:
# Extract service level code

p = re.compile('.*/.*/(.*)/.*/.*')

sl = df_energy_data_raw['ServiceAgreement'].str.extract(p)

df_energy_data_raw['Service'] = sl[0].str.strip()


## Correct timestamps

The timestamps in these files appear to be in local time. I discovered this when checking for dupes, and finding extra duplicates around e.g. Nov-07-2021 at 1-2am.

To ensure that the timestamps are aligned consistently, we need to adjust for DST. Otherwise, we'll see a funky offset when analyzing production by hour of day.

In [ ]:
# Check for dupes

df_dupe_check = df_energy_data_raw.groupby(['Service', 'IntervalStart'], as_index=False).agg(cnt_dupes=('IntervalStart','count'))

df_dupe_check = df_dupe_check[df_dupe_check['cnt_dupes'] != 1]

df_dupe_records = df_energy_data_raw.merge(df_dupe_check, on=['Service', 'IntervalStart'], how='inner').sort_values(['IntervalStart', 'Service'])

In [ ]:
df_dupe_records.groupby('dt_start').agg({'cnt_dupes':'sum'})

In [ ]:
display(df_dupe_records)

In [ ]:
df_energy_data_raw

There are 2 reasons for dupes- either the downloaded data files overlap, or we have duplicate timestamps during the "fall back" Daylight Savings Time. 

In order to get aligned solar output data, we need to 

In [ ]:
sh0 = df_energy_data_raw.shape

df_solar_data = df_energy_data_raw #[df_energy_data_raw['Service'] == 'NGEN']

# # Remove any errant header rows (one may stick around in the data b/c it's not a dupe of the header row)
# df_solar_data = df_solar_data[df_solar_data['ServiceAgreement'] != 'ServiceAgreement']

# # Now we don't need that column anymore!
# df_solar_data = df_solar_data.drop('ServiceAgreement', axis=1)

sh0 = df_solar_data.shape

df_solar_data = df_solar_data.drop_duplicates()

sh1 = df_solar_data.shape

cnt_dupes = sh0[0] - sh1[0]

print("Removed {:,} duplicate entries; {:,} left.".format(cnt_dupes, sh1[0]))

In [ ]:
# Drop ServiceAgreement (which contains some PID):

df_solar_data = df_solar_data.drop('ServiceAgreement', axis=1)

In [ ]:
df_solar_data

# Save Clean Dataset

In [ ]:
df_solar_data.to_csv(os.path.join(path, 'full_dataset.csv'))